In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Load images
img1 = cv2.imread('1674674548143.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('ishihara-18.png', cv2.IMREAD_GRAYSCALE)

# Display the original images
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(img1, cmap='gray')
ax[0].set_title('Imagen 1 Original')
ax[0].axis('off')
ax[1].imshow(img2, cmap='gray')
ax[1].set_title('Imagen 2 Original')
ax[1].axis('off')
plt.show()


In [ ]:
# Binarize the images using adaptive thresholding
img1_binary = cv2.adaptiveThreshold(img1, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)
img2_binary = cv2.adaptiveThreshold(img2, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

# Display the binarized images
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(img1_binary, cmap='gray')
ax[0].set_title('Imagen 1 Binarizada')
ax[0].axis('off')
ax[1].imshow(img2_binary, cmap='gray')
ax[1].set_title('Imagen 2 Binarizada')
ax[1].axis('off')
plt.show()


In [ ]:
def moore_boundary_tracing(binary_image):
   #Direcciones
    directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]
    
    # Punto inicial
    start_point = None
    for y in range(binary_image.shape[0]):
        for x in range(binary_image.shape[1]):
            if binary_image[y, x] == 255:  # White pixel
                start_point = (y, x)
                break
        if start_point is not None:
            break

    if start_point is None:
        return []

    # Inicial direccion
    boundary = [start_point]
    current_point = start_point
    prev_dir = 6

    while True:
        
        for i in range(8):
            next_dir = (prev_dir + i) % 8
            dy, dx = directions[next_dir]
            neighbor_y, neighbor_x = current_point[0] + dy, current_point[1] + dx
            if binary_image[neighbor_y, neighbor_x] == 255:  
                boundary.append((neighbor_y, neighbor_x))
                current_point = (neighbor_y, neighbor_x)
                #Direccion previa
                prev_dir = (next_dir + 4) % 8
                break

        # If we have looped back to the start, break
        if current_point == start_point:
            break

    return boundary

# Applica Moore's boundary tracing algorithm
boundary1 = moore_boundary_tracing(img1_binary)
boundary2 = moore_boundary_tracing(img2_binary)

# Visualize boundaries 
boundary_img1 = img1_binary.copy()
boundary_img2 = img2_binary.copy()

for y, x in boundary1:
    boundary_img1[y, x] = 127  # Gray for boundary
for y, x in boundary2:
    boundary_img2[y, x] = 127  # Gray for boundary

# Mostrar imagenes
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(boundary_img1, cmap='gray')
ax[0].set_title('Imagen 1 con Contorno')
ax[0].axis('off')
ax[1].imshow(boundary_img2, cmap='gray')
ax[1].set_title('Imagen 2 con Contorno')
ax[1].axis('off')
plt.show()


In [ ]:
def freeman_chain_code(boundary, connectivity=8):
    # Directions for connectivity-4 and connectivity-8
    directions_4 = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    directions_8 = [(0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1)]
    
    directions = directions_8 if connectivity == 8 else directions_4
    chain_code = []
    
    for i in range(len(boundary) - 1):
        current_point = boundary[i]
        next_point = boundary[i+1]
        direction = (next_point[0] - current_point[0], next_point[1] - current_point[1])
        chain_code.append(directions.index(direction))
    
    return chain_code

# Apply Freeman's chain code algorithm to the boundaries
chain_code1_4 = freeman_chain_code(boundary1, connectivity=4)
chain_code2_4 = freeman_chain_code(boundary2, connectivity=4)
chain_code1_8 = freeman_chain_code(boundary1, connectivity=8)
chain_code2_8 = freeman_chain_code(boundary2, connectivity=8)

chain_code1_4, chain_code2_4, chain_code1_8, chain_code2_8

In [ ]:
# Segment the original image using Moore's boundary mask
segmented_img1 = cv2.bitwise_and(img1, img1, mask=boundary_img1)
segmented_img2 = cv2.bitwise_and(img2, img2, mask=boundary_img2)

# Display the segmented images
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(segmented_img1, cmap='gray')
ax[0].set_title('Imagen 1 Segmentada con Máscara de Moore')
ax[0].axis('off')
ax[1].imshow(segmented_img2, cmap='gray')
ax[1].set_title('Imagen 2 Segmentada con Máscara de Moore')
ax[1].axis('off')
plt.show()

In [ ]:
def normalize_chain_code(chain_code):
    # Compute the first difference
    first_difference = [(chain_code[i] - chain_code[i - 1]) % 8 for i in range(len(chain_code))]

    # Find the rotation-invariant chain code with the minimum magnitude
    min_chain = min([first_difference[i:] + first_difference[:i] for i in range(len(first_difference))])

    return min_chain

# Normalize the Freeman chain codes
normalized_chain_code1_4 = normalize_chain_code(chain_code1_4)
normalized_chain_code2_4 = normalize_chain_code(chain_code2_4)
normalized_chain_code1_8 = normalize_chain_code(chain_code1_8)
normalized_chain_code2_8 = normalize_chain_code(chain_code2_8)

normalized_chain_code1_4, normalized_chain_code2_4, normalized_chain_code1_8, normalized_chain_code2_8
